This Python script creates a graphical user interface (GUI) using the `Tkinter` library to display Instagram post descriptions stored in a MySQL database. The script fetches random posts from the database, displays them in the GUI, and uses a pre-trained Named Entity Recognition (NER) model to identify locations and organizations in each post. Here’s a breakdown of the code:

### 1. **Model and Tokenizer Loading**:
   - The script loads a pre-trained NER model (`sigaldanilov/nerbart_model`) and tokenizer from Hugging Face, which are used to analyze text and detect named entities like locations and organizations.
   - The `label_names` list defines the NER labels (e.g., `B-LOC` for location beginning, `I-ORG` for inside an organization name).

### 2. **Database Connection**:
   - The `connect_to_database()` function connects to a MySQL database using the `mysql.connector` library. The database credentials are specified (host, user, password, and database name).
   - The database is assumed to have a table named `posts_new` containing Instagram post descriptions.

### 3. **Fetching Posts from the Database**:
   - The `fetch_random_posts()` function retrieves the first 1000 Instagram posts from the `posts_new` table and randomly selects 6 posts for display. The `LIMIT 1000` ensures that only the first 1000 rows are fetched, which is a performance optimization.
   - The selected posts are then randomly sampled to display in the GUI.

### 4. **NER Prediction**:
   - The `predict_ner()` function processes each post text and uses the NER model to predict named entities (locations, organizations, etc.).
   - It tokenizes the text, runs it through the model, and interprets the output to identify any detected locations or organizations.
   - The function handles subword tokens (e.g., "##tel" and "aviv") by merging them into a single entity (e.g., "Tel Aviv").

### 5. **Displaying Posts in the GUI**:
   - The `show_locations()` function is responsible for fetching posts from the database, clearing any previous content, and displaying the posts in a two-column layout.
   - Each post is displayed with a short version (first 50 characters), and clicking on a post opens a new window with full details.
   - The posts are displayed using `Tkinter` frames that mimic Instagram post cards, and an image placeholder is added (though it's just a gray square for now).

### 6. **Post Detail Window**:
   - When a post is clicked, the `show_post_details()` function opens a new window displaying the full post text and any identified locations.
   - If no locations are found, the window shows a message stating "No locations identified."

### 7. **GUI Layout**:
   - The main window contains a title and a button ("Fetch and Show Locations") that triggers the display of random Instagram posts.
   - The posts are shown in a scrollable frame, allowing the user to browse through them.
   - Each post is presented in a card-like layout, and the user can click on a post to view its full details in a new window.

### 8. **Key GUI Components**:
   - **`Tkinter.Tk()`**: The main window is created with this class.
   - **Scrollable Frame**: The posts are displayed in a scrollable frame created using a `Tkinter.Canvas` and a vertical scrollbar.
   - **Fetch Button**: A button is added to trigger the random fetching and displaying of posts.
   - **Two-Column Layout**: The posts are arranged in two columns using a grid layout.

### Summary of the Application:
1. **User Interaction**: 
   - The user clicks the "Fetch and Show Locations" button.
   - The application connects to a MySQL database, retrieves Instagram posts, and displays a random selection in the GUI.
2. **NER Location Detection**:
   - The NER model processes each post and identifies any locations or organizations mentioned.
   - Clicking a post shows full details along with any detected locations.
3. **Interactivity**: 
   - The user can scroll through posts, click on them to see details, and interact with the GUI easily.

This application is a practical tool for browsing Instagram posts from a database and analyzing them for location-related content using machine learning models.

In [ ]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector
import random
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "sigaldanilov/nerbart_model"  # Your actual model name
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the label names based on your model's 9 labels
label_names = ['O', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']

# Connect to the MySQL database
def connect_to_database():
    connection = mysql.connector.connect(
        host="localhost",  # e.g., "localhost"
        user="root",
        password="password",
        database="fp_db"
    )
    return connection

# Fetch random posts from the database
def fetch_random_posts(connection, limit=6):
    cursor = connection.cursor()
    cursor.execute(f"SELECT img_description FROM posts_new LIMIT 1000")  # Fetch first 1000 rows
    posts = cursor.fetchall()
    return random.sample(posts, limit)  # Get a random sample of 'limit' posts

# Function to predict locations from the text using the model
def predict_ner(text, model, tokenizer, label_names):
    tokenized_input = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    model.eval()
    with torch.no_grad():
        outputs = model(**tokenized_input)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2)

    predicted_labels = [
        label_names[idx] if idx < len(label_names) else 'O' for idx in predictions[0].numpy()
    ]

    tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'][0])

    locations = []
    current_location = ""
    inside_location = False
    for token, label in zip(tokens, predicted_labels):
        if token in ["[CLS]", "[SEP]"]:
            continue

        if token.startswith("##"):
            token = token[2:]
            current_location += token
        else:
            if inside_location and label not in ["B-LOC", "I-LOC", "B-ORG", "I-ORG"]:
                locations.append(current_location)
                current_location = ""
                inside_location = False

            if label in ["B-LOC", "I-LOC", "B-ORG", "I-ORG"]:
                token = token.lstrip("#")
                if not inside_location:
                    current_location = token
                    inside_location = True
                else:
                    current_location += " " + token

    if current_location and inside_location:
        locations.append(current_location)

    return locations

# Function to display a new window with full post details and location information
def show_post_details(post_text):
    # Create a new window for the detailed post information
    detail_window = tk.Toplevel(window)
    detail_window.title("Post Details")
    detail_window.geometry('600x400')

    # Predict locations in the post text
    locations = predict_ner(post_text, model, tokenizer, label_names)

    # Add post text in the new window
    post_label = tk.Label(detail_window, text=f"Post: {post_text}", wraplength=500, font=("Arial", 12))
    post_label.pack(pady=10)

    # Show location information
    if locations:
        locations_label = tk.Label(detail_window, text=f"Identified Locations: {', '.join(locations)}", font=("Arial", 10, "italic"))
        locations_label.pack(pady=10)
    else:
        locations_label = tk.Label(detail_window, text="No locations identified.", font=("Arial", 10, "italic"))
        locations_label.pack(pady=10)

# Function to display the posts in a two-column layout
def show_locations():
    connection = connect_to_database()
    posts = fetch_random_posts(connection)

    # Clear previous posts
    for widget in post_frame.winfo_children():
        widget.destroy()

    # Display posts in two columns
    for i, post in enumerate(posts):
        post_text = post[0]

        # Create a new frame for each post (like a card in Instagram)
        post_card = tk.Frame(post_frame, bd=1, relief="solid", padx=10, pady=10)
        post_card.grid(row=i//2, column=i%2, padx=10, pady=10, sticky="n")  # Two-column layout

        # Add an image placeholder (a square above each post)
        img_placeholder = tk.Frame(post_card, width=100, height=100, bg="gray")  # Placeholder square
        img_placeholder.pack(pady=5)  # Add some space around the placeholder

        # Show a short version of the post (first 50 characters)
        short_post_text = post_text[:50] + "..." if len(post_text) > 50 else post_text
        post_label = tk.Label(post_card, text=f"Post: {short_post_text}", anchor="w", justify="left", wraplength=250, font=("Arial", 12))
        post_label.pack(anchor="w")

        # Add click event to show full post details in a new window
        post_card.bind("<Button-1>", lambda e, text=post_text: show_post_details(text))

    connection.close()

# Create the main GUI window
window = tk.Tk()
window.title("NER Location Finder")
window.geometry('800x600')

# Create a frame to hold the title and center it
title_frame = tk.Frame(window)
title_frame.pack(pady=10)

# Add a title for the application and center it
title_label = tk.Label(title_frame, text="Randomly selected Instagram Posts", font=("Arial", 18, "bold"))
title_label.pack()

# Create a scrollable frame to hold all posts
canvas = tk.Canvas(window)
scrollbar = tk.Scrollbar(window, orient="vertical", command=canvas.yview)
post_frame = tk.Frame(canvas)

# Configure the scrollbar
post_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
)
canvas.create_window((0, 0), window=post_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)

canvas.pack(side="left", fill="both", expand=True)
scrollbar.pack(side="right", fill="y")

# Add a button to fetch posts and display identified locations
fetch_button = tk.Button(window, text="Fetch and Show Locations", command=show_locations, font=("Arial", 14))
fetch_button.pack(pady=10)

# Start the GUI event loop
window.mainloop()
